In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/ANN_Challenge1

In [ ]:
import tensorflow as tf
import numpy as np
import os
import random
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt

tfk = tf.keras
tfkl = tf.keras.layers

In [ ]:
# Load the dataset to be used for classification
!unzip split_dataset.zip

In [ ]:
# apply data augmentation

from tensorflow.keras.preprocessing.image import ImageDataGenerator

aug_data_gen = ImageDataGenerator(rotation_range=15,
                                        horizontal_flip=True,
                                        vertical_flip=True, 
                                        fill_mode='reflect',
                                        rescale=1/255.)

train_dataset = aug_data_gen.flow_from_directory(directory='split_dataset/training',
                                                 target_size=(96,96),
                                                 color_mode='rgb',
                                                classes=None,
                                               class_mode='categorical',
                                               batch_size=8,
                                               shuffle=True)

validation_dataset = aug_data_gen.flow_from_directory(directory='split_dataset/validation',
                                                 target_size=(96,96),
                                                 color_mode='rgb',
                                                classes=None, 
                                               class_mode='categorical',
                                               batch_size=8,
                                               shuffle=True)

In [ ]:
# inspect the target classes
print("Assigned labels")
print(train_dataset.class_indices)
print()
print("Training Target classes")
unique, counts = np.unique(train_dataset.classes, return_counts=True)
print(dict(zip(unique, counts)))
print()
print("Validation Target classes")
unique, counts = np.unique(validation_dataset.classes, return_counts=True)
print(dict(zip(unique, counts)))

In [ ]:
# import DenseNet169 as supernet
denseNet = tf.keras.applications.DenseNet169(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

# set the supernet as not trainable to perform Transfer Learning
denseNet.trainable = False

# build the entire network    
model = tf.keras.Sequential()
model.add(tf.keras.layers.Resizing(256,256,interpolation="lanczos5")) # resize the input image
model.add(tf.keras.layers.RandomContrast(0.3)) # apply random contrast to the input image
model.add(denseNet)
model.add(tf.keras.layers.GlobalAvgPool2D())
model.add(tf.keras.layers.Dense(units=1024, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(units=256, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(units=8, activation='softmax'))

# compile the model
model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(1e-5), metrics='accuracy')

# fit the model to the data
history = model.fit(
    x = train_dataset,
    batch_size = 32,
    epochs = 200,
    validation_data = validation_dataset,
    callbacks = [tfk.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=10, restore_best_weights=True)]
).history

In [ ]:
# fine tune with first 6 layers frozen
model.get_layer('densenet169').trainable = True
for i, layer in enumerate(model.get_layer('densenet169').layers[:6]):
  layer.trainable=False

model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(1e-5), metrics='accuracy')

history = model.fit(
    x = train_dataset,
    batch_size = 32,
    epochs = 200,
    validation_data = validation_dataset,
    callbacks = [tfk.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=10, restore_best_weights=True)]
).history

In [ ]:
model.save('SubmissionModel')